In [1]:
import os

In [2]:
%pwd

'/home/aman/Desktop/TextCraft/research'

In [3]:
os.chdir("../")

In [4]:
from TextCraft.constants import *
from TextCraft.utils.common import read_yaml, create_directories

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [22]:
@dataclass
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer: str
    model: str

In [23]:
class ConfigurationManager:
    def __init__(self , config= CONFIG_FILE_PATH, params = PARAMS_FILE_PATH):
        self.config = read_yaml(config)
        self.params = read_yaml(params)

        create_directories([self.config.artifacts_root])

    def get_data_transformation(self) -> DataTransformationConfig:
        config = self.config.data_tranformation
        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir = config.root_dir,
            data_path= config.data_path,
            tokenizer= config.tokenizer,
            model= config.model
        )
        return data_transformation_config

In [24]:
import os
from TextCraft.logging import logger
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from datasets import load_dataset, load_from_disk

In [25]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = GPT2Tokenizer.from_pretrained(config.tokenizer)
        self.tokenizer.pad_token = self.tokenizer.eos_token
        self.model = GPT2LMHeadModel.from_pretrained(config.model)
        self.model.resize_token_embeddings(len(self.tokenizer))
    def convert_example_to_features(self, example):
        tokenized_data = self.tokenizer(example["text"], padding = "max_length", truncation = True, max_length = 1024)
        tokenized_data["labels"] = tokenized_data["input_ids"].copy()
        return tokenized_data
    
    def convert(self):
        dataset = load_from_disk(self.config.data_path)
        dataset_pt = dataset.map(self.convert_example_to_features, batched=True, remove_columns=["text"])
        dataset_pt.save_to_disk(os.path.join(self.config.root_dir, "Dataset"))


In [26]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation()
    data_transformation = DataTransformation(config = data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2024-09-11 23:35:41,924: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-09-11 23:35:42,151: INFO: common: yaml file: params.yaml loaded successfully]
[2024-09-11 23:35:42,174: INFO: common: created directory at: artifacts]
[2024-09-11 23:35:42,180: INFO: common: created directory at: artifacts/data_transformation]


/home/aman/Desktop/TextCraft/Craft/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 1880/1880 [00:00<00:00, 20618.81 examples/s]
